In [ ]:
# 2.12.2020
# Novinky ve verzi 1-3
# Vytvoření nového DataFramu: B_listings_warehouse_df
# Prázdný cyklus zprovoznění
# Přidání timestamp sloupce


# 1.12.2020
# ZNOVU PROČTENÍ CELÉHO KÓDU A OVĚŘENÍ, ZDA VŠECHNO FUNGUJE, JAK MÁ
# VÝSLEDEK = vše v pořádku

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep
import re
from datetime import datetime

In [8]:
"""
I.
LOADING OF URLS TO SCRAPE
"""

A_new_urls_to_scrape_BEZREALITKY_df = pd.read_pickle("dataframes/A_new_urls_to_scrape_BEZREALITKY_df.pkl")
A_new_urls_to_scrape_BEZREALITKY_df

""


In [9]:
"""
II.
CONVERSION OF DATAFRAME COLUMN TO A LIST
"""

if not A_new_urls_to_scrape_BEZREALITKY_df.empty:
    urls_list = A_new_urls_to_scrape_BEZREALITKY_df[0].to_list()
    urls_list
else: 
    print("DATAFRAME", A_new_urls_to_scrape_BEZREALITKY_df, "je prázdný")
    urls_list = []

DATAFRAME Empty DataFrame
Columns: []
Index: [] je prázdný


In [ ]:
"""
# =========================TEST USAGE ONLY===========================

# INZERÁT POUZE SE ŠESTI POLOŽKAMI: https://www.bezrealitky.cz/nemovitosti-byty-domy/630655

urls_list = ['https://www.bezrealitky.cz/nemovitosti-byty-domy/630657']

# =========================TEST USAGE ONLY===========================
"""
#urls_list = ['https://www.bezrealitky.cz/nemovitosti-byty-domy/633816-nabidka-prodej-bytu-usti-nad-labem']
#urls_list = ['https://www.bezrealitky.cz/nemovitosti-byty-domy/633816-nabidka-prodej-bytu-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/633289-nabidka-prodej-bytu-u-nemocnice-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/633061-nabidka-prodej-bytu-jizerska-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/632358-nabidka-prodej-bytu-snp-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/632394-nabidka-prodej-bytu-pincova-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/410965-nabidka-prodej-bytu-tolsteho-usti-nad-labem-strekov', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/630298-nabidka-prodej-bytu-usti-nad-labem', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/629707-nabidka-prodej-bytu-stribrniky', 'https://www.bezrealitky.cz/nemovitosti-byty-domy/625085-nabidka-prodej-bytu-elisky-krasnohorske-usti-nad-labem']

In [10]:
"""
III.
MAKING LISTS TO BE FILLED WITH SCRAPPED DATA 
"""

# MAKING LISTS FOR THE OFFER FEATURES DATA
ids = []
dispozice = []
plochy = []
ceny = []
poplatky = []
vratna_kauce = []
mesta = []
mestske_casti = []
typy_vlastnictvi = []
typy_budovy = []
penb = []
vybavenost = []
podlazi = []
balkony = []
terasy = []
sklepy = []
lodzie = []
parkovani = []
vytahy = []
garaze  = []
k_dispozici_od = []
stari = []
provedeni = []
typ_vytapeni = []
podlazi_v_ramci_domu = []
urls = []
scraped_times = []

print("Lists created")

Lists created


In [11]:
"""
IV. DATA SCRAPING
"""

# TEST ON EMPTINESS OF URLS LIST
if len(urls_list) == 0: 
    print(urls_list, "je PRÁZDNÝ")
else:

    # lOADING URLS TO SCRAPE
    for url in urls_list:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
        r = requests.get(url, headers=headers)
        soup_inzeratu = BeautifulSoup(r.text, 'html.parser')

        # EXTRACTION OF ALL OFFER FEATURE CATEGORIES (steps 1-4)

        # 1. makes a list of 3 elements each containing tbody division
        offer_properties_raw = soup_inzeratu.find_all("tbody")
        # pick a second element of the listabove that contains all the offer features
        offer_properties_mixed = offer_properties_raw[0]
        # makes a list that contains all the offer features
        offer_properties_cleaned = offer_properties_mixed.find_all("td")

        # 2. MAKING A LIST CONTAINING RAW FEATURES DATA ()
        offer_properties_list = []
        for offer_property in offer_properties_cleaned:
            offer_properties_list.append(offer_property.text)

        # 3. MAKING A LIST OF FEATURE CATEGORIES
        list_of_feature_categories_raw = offer_properties_raw[0].find_all("th")
        list_of_feature_categories = []
        for category_name_raw in list_of_feature_categories_raw:
            list_of_feature_categories.append(category_name_raw.text[:-1])


        # 4. RAW FEATURES DATA CLEANING - 
        # INTERNET CATEGORY REMOVAL
        if list_of_feature_categories[0] == 'Internet':
            list_of_feature_categories.remove('Internet')
        # AD TEXTS REMOVAL
        for item in offer_properties_list:
            if item == "\nChci hypotéku\n\ni\n\n":
                offer_properties_list.remove("\nChci hypotéku\n\ni\n\n")
            if item == "\nZjistěte rychlost O2 internetu u vás doma.\n\n":
                offer_properties_list.remove('\nZjistěte rychlost O2 internetu u vás doma.\n\n')
            if item == 'Chci slevu na nábytek a elektro':
                offer_properties_list.remove('Chci slevu na nábytek a elektro')
            if item == 'Stěhování něco stojí. S naší půjčkou to zvládnete.':
                offer_properties_list.remove('Stěhování něco stojí. S naší půjčkou to zvládnete.')
            if item == 'Získejte družstevní byt s Buřinkou\n\ni\n\n':
                offer_properties_list.remove('Získejte družstevní byt s Buřinkou\n\ni\n\n')
        
        # ADD TIMESTAMP OF SCRAPING
        time_of_scraping = pd.Timestamp.now(tz=None)
        scraped_times.append(time_of_scraping)
        
        # FEATURES DATA DISTRIBUTION INTO OWN LISTS
        while True:
            # URL
            list_of_feature_categories.append("URL")
            offer_properties_list.append(url)
            urls.append(url)

            # ID
            x = 0
            ids.append(offer_properties_list[x])

            # DISPOZICE
            x = x+1
            if len(offer_properties_list) <= x:
                dispozice.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Dispozice":
                    dispozice.append(offer_properties_list[x])
                else:
                    dispozice.append("N/A")
                    x = x - 1

            # PLOCHA
            x = x+1
            if len(offer_properties_list) <= x:
                plochy.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Plocha":
                    plochy.append(offer_properties_list[x])            
                else:
                    plochy.append("N/A")
                    x = x - 1

            # CENA
            x = x+1
            if len(offer_properties_list) <= x:
                ceny.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Cena":
                    ceny.append(offer_properties_list[x])
                else:
                    ceny.append("N/A")
                    x = x - 1

            # POPLATKY
            x = x+1
            if len(offer_properties_list) <= x:
                poplatky.append("Missing_Category")
            else:
                if list_of_feature_categories[x] =="Poplatky":
                    poplatky.append(offer_properties_list[x])
                else:
                    poplatky.append("N/A")
                    x = x - 1

            # VRATNÁ KAUCE
            x = x+1
            if len(offer_properties_list) <= x:
                vratna_kauce.append("Missing_Category")
            else:
                if list_of_feature_categories[x] =="Vratná kauce":
                    vratna_kauce.append(offer_properties_list[x])
                else:
                    vratna_kauce.append("N/A")
                    x = x - 1

            # MĚSTO
            x = x+1
            if len(offer_properties_list) <= x:
                mesta.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Město":
                    mesta.append(offer_properties_list[x])
                else:
                    mesta.append("N/A")
                    x = x - 1

            # MĚSTSKÁ ČÁST
            x = x + 1
            if len(offer_properties_list) <= x:
                mestske_casti.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Městská část":
                    mestske_casti.append(offer_properties_list[x])
                else:
                    mestske_casti.append("N/A")
                    x = x - 1

            # TYP VLASTNICTVÍ
            x = x+1
            if len(offer_properties_list) <= x:
                typy_vlastnictvi.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Typ vlastnictví":
                    typy_vlastnictvi.append(offer_properties_list[x])
                else: 
                    typy_vlastnictvi.append("N/A")
                    x = x-1

            # TYP BUDOVY
            x = x+1
            if len(offer_properties_list) <= x:
                typy_budovy.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Typ budovy":
                    typy_budovy.append(offer_properties_list[x])
                else: 
                    typy_budovy.append("N/A")
                    x = x-1

            # PENB
            x = x+1
            if len(offer_properties_list) <= x:
                penb.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "PENB":
                    penb.append(offer_properties_list[x])
                else: 
                    penb.append("N/A")
                    x = x-1

            # VYBAVENOST
            x = x+1
            if len(offer_properties_list) <= x:
                vybavenost.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Vybavenost":
                    vybavenost.append(offer_properties_list[x])
                else: 
                    vybavenost.append("N/A")
                    x = x-1

            # PODLAŽÍ
            x = x+1
            if len(offer_properties_list) <= x:
                podlazi.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Podlaží":
                    podlazi.append(offer_properties_list[x])
                else: 
                    podlazi.append("N/A")
                    x = x-1

            # BALKÓN
            x = x+1
            if len(offer_properties_list) <= x:
                balkony.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Balkón":
                    balkony.append(offer_properties_list[x])
                else: 
                    balkony.append("N/A")
                    x = x-1

            # TERASA
            x = x+1
            if len(offer_properties_list) <= x:
                terasy.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Terasa":
                    terasy.append(offer_properties_list[x])
                else: 
                    terasy.append("N/A")
                    x = x-1

            # SKLEP
            x = x+1
            if len(offer_properties_list) <= x:
                sklepy.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Sklep":
                    sklepy.append(offer_properties_list[x])
                else: 
                    sklepy.append("N/A")
                    x = x-1

            # LODŽIE 
            x = x+1
            if len(offer_properties_list) <= x:
                lodzie.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Lodžie":
                    lodzie.append(offer_properties_list[x])
                else: 
                    lodzie.append("N/A")
                    x = x-1

            # PARKOVÁNÍ
            x = x+1
            if len(offer_properties_list) <= x:
                parkovani.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Parkování":
                    parkovani.append(offer_properties_list[x])
                else: 
                    parkovani.append("N/A")
                    x = x-1

            # VÝTAH
            x = x+1
            if len(offer_properties_list) <= x:
                vytahy.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Výtah":
                    vytahy.append(offer_properties_list[x])
                else: 
                    vytahy.append("N/A")
                    x = x-1

            # GARÁŽ
            x = x+1
            if len(offer_properties_list) <= x:
                garaze.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Garáž":
                    garaze.append(offer_properties_list[x])
                else: 
                    garaze.append("N/A")
                    x = x-1

            # K DISPOZICI OD
            x = x+1
            if len(offer_properties_list) <= x:
                k_dispozici_od.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "K dispozici od":
                    k_dispozici_od.append(offer_properties_list[x])
                else: 
                    k_dispozici_od.append("N/A")
                    x = x-1

            # STÁŘÍ
            x = x+1
            if len(offer_properties_list) <= x:
                stari.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Stáří":
                    stari.append(offer_properties_list[x])
                else: 
                    stari.append("N/A")
                    x = x-1

            # PROVEDENÍ
            x = x+1
            if len(offer_properties_list) <= x:
                provedeni.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Provedení":
                    provedeni.append(offer_properties_list[x])
                else: 
                    provedeni.append("N/A")
                    x = x-1

            #TYP VYTÁPĚNÍ
            x = x+1
            if len(offer_properties_list) <= x:
                typ_vytapeni.append("Missing_Category")
            else:
                if list_of_feature_categories[x] == "Typ vytápění":
                    typ_vytapeni.append(offer_properties_list[x])
                else: 
                    typ_vytapeni.append("N/A")
                    x = x-1

            # PODLAŽÍ V RÁMCI DOMU
            x = x+1
            if len(offer_properties_list) <= x:
                podlazi_v_ramci_domu.append("Missing_Category")
                break
            else:
                if list_of_feature_categories[x] == "Podlaží v rámci domu":
                    podlazi_v_ramci_domu.append(offer_properties_list[x])
                else: 
                    podlazi_v_ramci_domu.append("N/A")
                    x = x-1
                break
            
        #print("Délka seznamu features pro:",url,"je:",len(offer_properties_list)) # ověření délky seznamu, zda obsahuje všech 18 položek
        print(url,  "= HOTOVO")
        sleep(5)
    print("CELÉ HOTOVO !!!")

[] je PRÁZDNÝ


In [12]:
"""
KONTROLNÍ VÝPISY
"""

# TEST ON EMPTINESS OF URLS LIST
if len(urls_list) == 0: 
    print(urls_list, "je PRÁZDNÝ")
else:
    # Názvy katergorií inzerátu (PŘED odstraněním nežádoucích položek)
    print(len(list_of_feature_categories_raw))
    list_of_feature_categories_raw
    
    # názvy katergorií inzerátu (PO odstranění nežádoucích položek)
    print(len(list_of_feature_categories))
    list_of_feature_categories

    #  - položky inzerátu
    print(len(offer_properties_list))
    offer_properties_list

    zip_iterator = zip(list_of_feature_categories, offer_properties_list)
    features_dictionary = dict(zip_iterator)
    for key, value in features_dictionary.items():
        print(key,":",value)

    # -??????? ale nejsem si jistý :))))

    list_of_lists=[
        ids,
        dispozice,
        plochy,
        ceny,
        poplatky,
        vratna_kauce,
        mesta,
        mestske_casti,
        typy_vlastnictvi,
        penb,
        vybavenost,
        podlazi,
        balkony,
        terasy,
        sklepy,
        lodzie,
        parkovani,
        vytahy,
        garaze,
        k_dispozici_od,
        stari,
        provedeni,
        typ_vytapeni,
        podlazi_v_ramci_domu,
        urls,
        scraped_times,
    ]
    for x in list_of_lists:
        print(x, sep="/n")

[] je PRÁZDNÝ


In [13]:
"""
V.
LIST WITH NAMES OF COLUMNS
"""

property_names_list=['Číslo inzerátu',
                     'Dispozice',
                     'Plocha',
                     'Cena',
                     'Poplatky',
                     'Vratná kauce',
                     'Město',
                     'Městská část',
                     'Typ vlastnictví',
                     'Typ budovy',
                     'PENB',
                     'Vybavenost',
                     'Podlaží',
                     'Balkón',
                     'Terasa',
                     'Sklep',
                     'Lodžie',
                     'Parkování',
                     'Výtah',
                     'Garáž',
                     'K dispozici od',
                     'Stáří',
                     'Provedení',
                     'Typ vytápění',
                     'Podlaží v rámci domu',
                     'url',
                     'scraped_times'
                    ]

In [14]:
"""
VI.
MAKING A DATAFRAME OF NEWEST LISTINGS
"""

# POZOR: ani jeden ze seznamů nesmí zůstat úplně prázdný, jinak se dataframe nebude chtít vytvořit
# POZOR: ani jeden ze seznamů nesmí být částečně prádný - velmi matoucí

# TEST ON EMPTINESS OF URLS LIST
if len(urls_list) == 0: 
    B_new_listings_BEZREALITKY_df = pd.DataFrame() 
else:
    B_new_listings_BEZREALITKY_df = pd.DataFrame(zip(ids,dispozice,plochy,ceny,poplatky, vratna_kauce, mesta,mestske_casti,typy_vlastnictvi,typy_budovy,penb,vybavenost,podlazi,balkony,terasy,sklepy,lodzie,parkovani,vytahy,garaze,k_dispozici_od,stari, provedeni, typ_vytapeni,podlazi_v_ramci_domu,urls,scraped_times),
                            columns=property_names_list)
    B_new_listings_BEZREALITKY_df

In [3]:
"""
VII. LOADING OF LISTINGS WAREHOUSE DataFrame (BEZREALITKY)
"""
B_listings_warehouse_BEZREALITKY_df = pd.read_pickle("dataframes/B_listings_warehouse_BEZREALITKY_df.pkl")
B_listings_warehouse_BEZREALITKY_df

,Číslo inzerátu,Dispozice,Plocha,Cena,Poplatky,Vratná kauce,Město,Městská část,Typ vlastnictví,Typ budovy,...,Parkování,Výtah,Garáž,K dispozici od,Stáří,Provedení,Typ vytápění,Podlaží v rámci domu,url,scraped_times
0,633816,2+kk,48 m²,750.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Družstevní,Panel,...,Ne,Ano,Ne,01. 12. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:34:53.638612
1,630298,1+1,47 m²,10.000.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,01. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:34:59.079067
2,625085,3+1,92 m²,2.200.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,07. 09. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:04.649770
3,200547,1+1,36 m²,910.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 1970,N/A,standardní: běžné materiály a technologie,ústřední dálkové,přízemí,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:10.459787
4,640687,3+1,65 m²,1.900.000 Kč,N/A,N/A,Ústí nad Labem,Střekov,Osobní,Panel,...,Ne,Ano,Ne,01. 01. 1970,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:16.303273
5,640666,2+1,65 m²,1.490.000 Kč,N/A,N/A,Ústí nad Labem,Dobětice,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:22.099996
6,639650,3+1,76 m²,1.100.000 Kč,N/A,N/A,Ústí nad Labem,Mojžíř,Osobní,Panel,...,Ne,Ano,Ne,15. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:27.801042
7,639500,1+1,35 m²,1.170.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 1970,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:33.635716
8,639734,3+1,80 m²,1.500.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,01. 12. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:39.463147
9,634864,3+1,78 m²,1.800.000 Kč,N/A,N/A,Ústí nad Labem,Dobětice,Osobní,Panel,...,Ne,Ano,Ne,02. 02. 2021,N/A,N/A,ústřední dálkové,poslední podlaží,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:45.275786


In [15]:
"""
VIII. SLOUČENÍ DATAFRAMŮ POMOCÍ METODY CONCAT() 
==> sloučí dataframy pod sebe

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
"""
B_listings_warehouse_BEZREALITKY_df = pd.concat([B_listings_warehouse_BEZREALITKY_df, B_new_listings_BEZREALITKY_df])
B_listings_warehouse_BEZREALITKY_df.reset_index(drop=True, inplace=True)
B_listings_warehouse_BEZREALITKY_df

,Číslo inzerátu,Dispozice,Plocha,Cena,Poplatky,Vratná kauce,Město,Městská část,Typ vlastnictví,Typ budovy,...,Parkování,Výtah,Garáž,K dispozici od,Stáří,Provedení,Typ vytápění,Podlaží v rámci domu,url,scraped_times
0,633816,2+kk,48 m²,750.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Družstevní,Panel,...,Ne,Ano,Ne,01. 12. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:34:53.638612
1,630298,1+1,47 m²,10.000.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,01. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:34:59.079067
2,625085,3+1,92 m²,2.200.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,07. 09. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:04.649770
3,200547,1+1,36 m²,910.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 1970,N/A,standardní: běžné materiály a technologie,ústřední dálkové,přízemí,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:10.459787
4,640687,3+1,65 m²,1.900.000 Kč,N/A,N/A,Ústí nad Labem,Střekov,Osobní,Panel,...,Ne,Ano,Ne,01. 01. 1970,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:16.303273
5,640666,2+1,65 m²,1.490.000 Kč,N/A,N/A,Ústí nad Labem,Dobětice,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:22.099996
6,639650,3+1,76 m²,1.100.000 Kč,N/A,N/A,Ústí nad Labem,Mojžíř,Osobní,Panel,...,Ne,Ano,Ne,15. 01. 2021,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:27.801042
7,639500,1+1,35 m²,1.170.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Panel,...,Ne,Ne,Ne,01. 01. 1970,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:33.635716
8,639734,3+1,80 m²,1.500.000 Kč,N/A,N/A,Ústí nad Labem,N/A,Osobní,Cihla,...,Ne,Ne,Ne,01. 12. 2020,N/A,N/A,N/A,N/A,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:39.463147
9,634864,3+1,78 m²,1.800.000 Kč,N/A,N/A,Ústí nad Labem,Dobětice,Osobní,Panel,...,Ne,Ano,Ne,02. 02. 2021,N/A,N/A,ústřední dálkové,poslední podlaží,https://www.bezrealitky.cz/nemovitosti-byty-do...,2020-12-02 15:35:45.275786


In [4]:
"""
IX. EXPORT OF UPDATED LISTINGS WAREHOUSE DataFrame
"""

B_listings_warehouse_BEZREALITKY_df.to_pickle("dataframes/B_listings_warehouse_BEZREALITKY_df.pkl")

In [17]:

"""
EXPORT OF DATAFRAME OF NEWEST LISTINGS (FOR THE NEXT PROCESSING)
"""

B_new_listings_BEZREALITKY_df.to_pickle("dataframes/B_new_listings_BEZREALITKY_df.pkl")

In [ ]:
# 499 řádků kódu včetně mezer a komentářů A KONTROLNÍCH ÚSEKŮ (spočítáno 4.11.2020)